In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa, utils
from cryptography import exceptions
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

In [ ]:
def generate_key_pair():
    # Generate RSA private key (contains public key information)
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    # Extract the public key from the private key
    public_key = private_key.public_key()
    return private_key, public_key


In [ ]:
def encrypt_message(message: str, public_key: rsa.RSAPublicKey):
    """Encrypt a message with RSA-OAEP (asymmetric encryption)."""
    ciphertext = public_key.encrypt(
        message.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

In [ ]:
def decrypt_message(ciphertext: bytes, private_key: rsa.RSAPrivateKey):
    """Decrypt a message with RSA-OAEP."""
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode('utf-8')

In [ ]:
def sign_message(message: str, private_key: rsa.RSAPrivateKey):
    # Create message digest using SHA-256
    digest = hashes.Hash(hashes.SHA256(), default_backend())
    digest.update(message.encode('utf-8'))
    message_digest = digest.finalize()

    # Sign the digest with private key
    signature = private_key.sign(
        message_digest,
        padding.PKCS1v15(),
        utils.Prehashed(hashes.SHA256())
    )
    return signature

In [ ]:
def verify_signature(message: str, signature: bytes, public_key: rsa.RSAPublicKey):
    # Create message digest using SHA-256
    digest = hashes.Hash(hashes.SHA256(), default_backend())
    digest.update(message.encode('utf-8'))
    message_digest = digest.finalize()

    # Verify the signature with public key
    try:
        public_key.verify(
            signature,
            message_digest,
            padding.PKCS1v15(),
            utils.Prehashed(hashes.SHA256())
        )
        return True
    except exceptions.InvalidSignature:
        return False


In [ ]:
# Generate keys
private_key, public_key = generate_key_pair()
print(private_key.private_bytes())

message = "Hello, world!"

# Encryption/Decryption demo

# ciphertext = encrypt_message(message, public_key)
# print(f"Cipher Text:{ciphertext.hex()}")
# # # print(bytes_to_pem(ciphertext))

# decrypted = decrypt_message(ciphertext, private_key)
# print(f"Decrypted: {decrypted}")

# Digital signature demo

# signature = sign_message(message, private_key)

# print(f"Original: {message}")
# print(f"Signature: {signature.hex()}")

# print(f"Signature valid: {verify_signature(message, signature, public_key)}")

----
##### PEM encoding/decoding

In [ ]:
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    PublicFormat,
    NoEncryption,
    BestAvailableEncryption
)
import base64

def bytes_to_pem(data: bytes, label:str="CIPHER_TEXT") -> str:
    """Convert bytes to PEM format with custom headers."""
    header = f"-----BEGIN {label}-----\n"
    footer = f"\n-----END {label}-----"
    # Base64 encode and split into 64-character lines
    b64 = base64.b64encode(data).decode('ascii')
    b64_lines = [b64[i:i+64] for i in range(0, len(b64), 64)]
    return header + '\n'.join(b64_lines) + footer

def pem_to_bytes(pem_str: str) -> bytes:
    """Extract bytes from PEM string."""
    # Remove headers/footers and whitespace
    lines = [line.strip() for line in pem_str.splitlines()
             if line.strip() and not line.strip().startswith('-----')]
    return base64.b64decode(''.join(lines))

# Example usage
# if __name__ == "__main__":
#     import base64

#     ciphertext = b'\x12\x34\x56\x78\x9a\xbc\xde\xf0' * 10

#     # Convert to PEM
#     pem_msg = bytes_to_pem(ciphertext, "ENCRYPTED DATA")
#     print("PEM Encoded:")
#     print(pem_msg)

#     # Convert back
#     decoded = pem_to_bytes(pem_msg)
#     print("\nDecoded matches original:", decoded == ciphertext)

In [ ]:
from cryptography.hazmat.primitives import serialization

# Save private key
def save_private_key(private_key, file_name = 'private_key.pem'):
    pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open(file_name, 'wb') as f:
        f.write(pem)

# Save public key
def save_public_key(public_key, file_name = 'public_key.pem'):

    pem = public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    with open(file_name, 'wb') as f:
        f.write(pem)


In [ ]:
save_public_key(public_key)
save_private_key(private_key)

In [ ]:
def load_private_key(filename='private_key.pem'):
    with open(filename, 'rb') as f:
        private_key = serialization.load_pem_private_key(
            f.read(),
            None,
            backend=default_backend()
        )
    return private_key

def load_public_key(filename):
    with open(filename, 'rb') as f:
        public_key = serialization.load_pem_public_key(
            f.read(),
            backend=default_backend()
        )
    return public_key